In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from tardis import run_tardis
import tardis
import plotly.graph_objects as go
from collections import defaultdict
from pylab import *

In [3]:
cmap = cm.get_cmap('jet', 20)
colors = []
for i in range(cmap.N):
    rgb = cmap(i)[:3]
    colors.append(matplotlib.colors.rgb2hex(rgb))

In [4]:
marker_colors = ["#958aff","#ff8b85","#5cff74"]
marker_line_colors = ['#27006b', '#800000', '#00801c']
marker_colors = ['#636EFA', '#EF553B', '#00CC96']
luminosities = ["Emitted", "Absorbed", "Requested"]
X = list(range(1,21))

In [5]:
import plotly.graph_objects as go
fig = go.FigureWidget().set_subplots(
    rows = 3,
    cols = 2,
    specs = [
        [{}, {}],
        [{"colspan": 2}, None],
        [{"colspan": 2}, None]
    ],
    row_heights=[0.55, 0.4,0.1],
    vertical_spacing = 0.07
)

marker_colors = ["#958aff","#ff8b85","#5cff74"]
marker_line_colors = ['#27006b', '#800000', '#00801c']
marker_colors = ['#636EFA', '#EF553B', '#00CC96']
luminosities = ["Emitted", "Absorbed", "Requested"]
X = list(range(1,21))


for luminosity, marker_color, marker_line_color in zip(luminosities, marker_colors, marker_line_colors):
    fig.add_scatter(
        name = luminosity + "<br>Luminosity", 
        mode='lines+markers', 
        row = 2,
        col = 1,
        marker_color=marker_color, 
        marker_line_color=marker_line_color,
        legendgroup = luminosity, 
        marker_line_width=1.5, 
        opacity=0.6
    )
fig.add_scatter(
    name = "Residual<br>Luminosity",
    row = 3, 
    col = 1,
    hovertext="text",
    marker_color='rgb(158,202,225)', 
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5, 
    mode='lines+markers',
    opacity=0.7

)

fig.add_scatter(row = 1, col = 1)
fig.add_scatter(row = 1, col = 2)

fig.update_layout(
    height = 1000,
    yaxis = {
        "range": [9000, 14000],
        "tickformat": "g",
        "title": r"$T_{rad}\ [K]$"
    },
    yaxis2 = {
        "title": r"$W$"
    },
    yaxis3 = {
        "title": r"$\mbox{Luminosity}~(erg~sec^{-1})$",
        "exponentformat": "e",
    },
    yaxis4 = {
        "title": r"$~~\mbox{Residual}\\\mbox{Luminosity(%)}$", 
        "exponentformat": "e",
        "title_font_size": 13,
    },
    
    xaxis = {
        "title": r"$Shell~~Velocity$",
        "tickformat": "g"
    },
    xaxis2 = {
        "title": r"$Shell~~Velocity$",
        "tickformat": "g",
        "matches": "x"
    },
    xaxis3 = {
        "dtick": 2,
        "range": [0, 21],
    },
    xaxis4 = {
        "title": r"$\mbox{Iteration Number}$", 
        "matches": "x3",
        "dtick": 2,
        "range": [0, 21],
    },
    
    
    
)



FigureWidget({
    'data': [{'legendgroup': 'Emitted',
              'marker': {'color': '#636EFA', 'line': {'…

In [6]:
i=0
ic = 0
traces = defaultdict(list)

In [7]:
def update_plots(sim):
    global i, ic, traces
    i+=2
    
    shell_velocity = [item/100000 for item in sim.model.velocity.value.tolist()]
    emitted_luminosity = sim.runner.calculate_emitted_luminosity(
        sim.luminosity_nu_start, sim.luminosity_nu_end
    )
    reabsorbed_luminosity = sim.runner.calculate_reabsorbed_luminosity(
        sim.luminosity_nu_start, sim.luminosity_nu_end
    )

    traces["Emitted"].append(emitted_luminosity.value)
    traces["Absorbed"].append(reabsorbed_luminosity.value)
    traces["Requested"].append(sim.luminosity_requested.value)
    

    
    
    with fig.batch_update():
        for index, luminosity in zip(range(3), luminosities):
            fig.data[index].x = X
            fig.data[index].y = traces[luminosity]
            fig.data[index].hovertemplate = '<b>%{y:.2g}</b>'+'<br>at X = %{x}<br>'
    
        y = [((emitted-requested)*100)/requested for emitted, requested in zip(traces["Emitted"],traces["Requested"])]
        fig.data[3].x = X
        fig.data[3].y = y
        fig.data[3].hovertemplate='Residual Luminosity: %{y:.2f}% at X = %{x:,.0f}<extra></extra>'
        
        fig.add_scatter(
            x = shell_velocity,
            y = sim.model.t_rad.value.tolist(), 
            line_color = colors[ic], 
            row = 1, 
            col = 1, 
            name = ic+1,
            legendgroup=f"group-{ic}", 
            showlegend = False,
        )


        fig.add_scatter(
            x = shell_velocity,
            y = sim.model.w.tolist(), 
            line_color= colors[ic], 
            row = 1, 
            col = 2, 
            legendgroup=f"group-{ic}",
            name = ic+1,
        )
    ic+=1

    

In [8]:
sim = run_tardis('tardis_example.yml', 
                 simulation_callbacks=[[update_plots]])

[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/atharva/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
[py.warnings         ][WARNING]  /home/atharva/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning:

divide by zero encountered in double_scalars

 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Lumino

[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/atharva/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning:

divide by zero encountered in double_scalars

 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 erg / s Luminosity requested = 1.05928e+43

[tardis.simulation.base][INFO   ]  t_inner 11175.750 K -- next t_inner 10152.603 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/atharva/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning:

divide by zero encountered in double_scalars

 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.75331e+42 erg / s Luminosity absorbed = 2.81319e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11437.188130  10637.774893  0.496749  0.452668
	5      11571.106856  11002.647927  0.196714  0.166197
	10     11378.137918  10832.042108  0.120954  0.101284
	15     10957.211814  10350.602742  0.092896  0